In [21]:
# The model definition provided
import sklearn as sk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import math
import tensorflow as tf
import tensorflow.keras as k
from keras import backend as K
# from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Conv1D, Input, Dense, concatenate, Flatten, Add, Activation, BatchNormalization, Reshape
from tensorflow.keras import Model
import sys

In [22]:
import pandas as pd
import numpy as np

data1 = pd.read_csv("GSE2034-Normal-train.txt", sep="\t")
data2 = pd.read_csv("GSE2034-Tumor-train.txt", sep="\t")
# data format:
# ----------- HEADER LINE ------------
# 2 s1g1 s2g1 s3g1 ... sng1
# 9 s1g2 s2ge s3g2 ... sng2
#          ...........
# k s1g12634 s2g12634 ... sng12634
# use data1.head() to see
data1 = data1.values[:,1:].transpose()
data2 = data2.values[:,1:].transpose()
data1Label = np.zeros(len(data1)).reshape((-1, 1))
data2Label = np.ones(len(data2)).reshape((-1, 1))
x = np.concatenate((data1, data2))
y = np.concatenate((data1Label, data2Label))

In [53]:
dim = 10
def res1d(x1, layers, kernel=(3,), strides=1,):    
    ConvLayer = k.layers.Conv1D        
    normalizer = BatchNormalization

    c1 = x1        
    c1 = ConvLayer(layers, kernel_size=kernel, use_bias=0, strides=strides, padding='same')(c1)        
    c1 = normalizer(gamma_initializer='zeros')(c1)
    
    o1 = x1
    if strides > 1 or K.int_shape(x1)[-1] != layers:
        if strides > 1:
            o1 = k.layers.ZeroPadding1D((0, strides-1))(o1)
            o1 = k.layers.AveragePooling1D(strides)(o1)
        o1 = ConvLayer(layers, kernel_size=1, use_bias=0, padding='same')(o1)
        o1 = normalizer()(o1)
    print_op1 = tf.print("o1:", o1, output_stream=sys.stdout, summarize=-1)
    print_op2 = tf.print("c1:", c1, output_stream=sys.stdout, summarize=-1)
    with tf.control_dependencies([print_op1, print_op2]):
        v1 = Add()([c1, o1])
    v1 = Activation('relu')(v1)
    return v1

def build():
    inp = Input(shape=(dim,)) # length of input
    
    v1 = k.layers.Reshape((-1, 1))(inp)
    v1 = res1d(v1, 4, 3, strides=2)

    model = k.models.Model(inputs=inp, outputs=v1)
    return model

In [54]:
model = build()
data1.shape
inputarray = np.array([[1,2,3,4,5,6,7,8,9,10]])
inputarray.shape

(1, 10)

In [55]:
model.compile(optimizer=k.optimizers.Adam(lr=0.0001), metrics=['acc'], loss='binary_crossentropy')
model.predict(inputarray)
pass

c1: [[[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]
o1: [[[-0.607571423 1.4711225 -0.235185936 1.0158174]
  [-1.41766667 3.43261933 -0.548767209 2.37024045]
  [-2.22776175 5.39411592 -0.862348437 3.7246635]
  [-3.03785706 7.35561275 -1.17592967 5.07908678]
  [-3.84795237 9.31711 -1.48951089 6.43351]]]
